In [ ]:
# 引入人臉識別庫dlib
import dlib

In [ ]:
# 引入影象處理庫OpenCV
import cv2

In [ ]:
# Cascade path
cascPath = "data/haarcascades/haarcascade_eye.xml"

In [ ]:
# Create the haar cascade
eye_cascade = cv2.CascadeClassifier(cascPath)

In [ ]:
# 使用dlib庫提供的人臉提取器
detector = dlib.get_frontal_face_detector()

In [ ]:
# 檔名流水號
face_filename = 1

In [ ]:
# 讀取影片，並抓取幀畫面圖片中的人臉部位圖，將其轉為pgm格式，儲存於指定位置
def videoToPng(videoPath, saveDir):
    
    # 讀取影片
    cap = cv2.VideoCapture(videoPath)
    
    # 影片幀技術間隔頻率
    timeF = 10 
    
    # 計數器
    c=1
    
    # 循環讀取影片
    while(True):
        
        # 從VideoCapture擷取一張影像
        ret, frame = cap.read() 
        
        # 每隔頻率進行操作
        if(c%timeF == 0):
            detectFaceAndSave(frame, saveDir)
        
        # 計數器遞增
        c = c + 1
        
        # 刷新
        cv2.waitKey(1)
        
        # 當最後讀取不到幀時，跳出迴圈
        if ret == False:
            print('videoToPng job finish')
            break
            
    # 釋放攝影機
    cap.release()
    
    

In [ ]:
# 抓取圖片中的人臉部位圖，將其轉為pgm格式，儲存於指定位置
def detectFaceAndSave(img, saveDir):
    
    # 將來源圖轉為灰度圖
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # 返回臉的資訊
    faces = detector(gray)
    
    # 當人臉有一個以上時，則進入條件式
    if len(faces) > 0:
        
        # 遍尋每張人臉
        for face in faces:
            
            # 擷取圖片中，人臉的部位圖片
            crop_img_gray = gray[face.top():face.bottom(),face.left():face.right()]
            
            # 偵測人臉的眼睛
            eyes = eye_cascade.detectMultiScale(
                crop_img_gray,
                scaleFactor=1.1, #表示在前後兩次相繼的掃描中，搜索窗口的比例係數。默認為1.1即每次搜索窗口依次擴大10%
                minNeighbors=5, #表示構成檢測目標的相鄰矩形的最小個數(默認為3個)
                minSize=(30, 30),#限制得到的目標區域的最小範圍          
            )
            
            # 使用全域變數
            global face_filename
            
            # 當人臉有兩個眼睛時，則進入條件式
            if(len(eyes) == 2):
                
                # 字串格式化處理儲存位置以及檔案名稱
                name = '{}{}.pgm'.format(saveDir, face_filename)
                print(name)
                
                # 統一圖片大小
                crop_img_gray = cv2.resize(crop_img_gray, (400, 400))
                
                # 寫入圖片
                cv2.imwrite(name, crop_img_gray)
                
                # 檔名流水號遞增
                face_filename+= 1
                
                

In [ ]:
# 程式進入點
if __name__ == '__main__':
    
    # 影片檔來源路徑
    # videoPath = 'data/video/Obama.mp4'
    videoPath = 'data/video/TT_TZUYU.mp4'
    
    # 儲存圖片檔路徑
    # saveDir = 'data/image/train/Obama/'
    saveDir = 'data/image/train/TZUYU/'

    videoToPng(videoPath, saveDir)